In [2]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import SnowballStemmer

# Descargar el corpus de stopwords y punkt si no está disponible
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\erick\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\erick\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
def create_inverted_index(directory):
    index = {}
    stemmer = SnowballStemmer('english')
    
    for root, dirs, files in os.walk(directory):
        for file in files[:100]:  # Take only the first 100 files
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read().lower()  # Convert to lowercase
                tokens = word_tokenize(content)
                tokens = [stemmer.stem(token) for token in tokens if token not in stopwords.words('english') and token not in string.punctuation]
                for word in tokens:
                    if word in index:
                        index[word].append(file_path)
                    else:
                        index[word] = [file_path]
    return index

In [4]:
# Directory where the documents are located
directory = "../../data"

inverted_index = create_inverted_index(directory)
print("Inverted index created.")

Inverted index created.


In [5]:
# Convert the inverted index into a dataframe
df_index = pd.DataFrame([(word, files) for word, files in inverted_index.items()], columns=['Word', 'Files'])

# Save the inverted index as a CSV file
df_index.to_csv('inverted_index.csv', index=False)

print("Inverted index saved as 'inverted_index.csv'.")

Inverted index saved as 'inverted_index.csv'.


In [14]:
# Cargar el archivo CSV generado
df_index = pd.read_csv('inverted_index.csv')

# Mostrar el DataFrame cargado
print(df_index.head())

        Word                                              Files
0       ﻿the  ['../../data\\pg100.txt', '../../data\\pg100.t...
1    project  ['../../data\\pg100.txt', '../../data\\pg100.t...
2  gutenberg  ['../../data\\pg100.txt', '../../data\\pg100.t...
3      ebook  ['../../data\\pg100.txt', '../../data\\pg100.t...
4    complet  ['../../data\\pg100.txt', '../../data\\pg100.t...


In [7]:
def process_query(query, inverted_index):
    query_parts = query.split()
    result = None
    
    while query_parts:
        token = query_parts.pop(0)
        if token == 'AND':
            operand1 = result
            operand2 = set(inverted_index[query_parts.pop(0)])
            result = operand1.intersection(operand2)
        elif token == 'OR':
            operand1 = result
            operand2 = set(inverted_index[query_parts.pop(0)])
            result = operand1.union(operand2)
        elif token == 'NOT':
            operand1 = result
            operand2 = set(inverted_index[query_parts.pop(0)])
            result = operand1.difference(operand2)
        else:
            result = set(inverted_index[token])
    
    return result

In [20]:
query = "hi AND bye"
result = process_query(query, inverted_index)

# Formatear y mostrar resultados de manera más elegante
formatted_results = [os.path.basename(file_path) for file_path in result]
print("Documentos encontrados:")
for file_name in formatted_results:
    print(file_name)

Documentos encontrados:
pg4300.txt
pg2160.txt
pg39407.txt
pg98.txt
